In [2]:
import os
import pandas as pd

# 1. Load your test data
test_df = pd.read_excel('test2.xlsx')
expected_ids = set(test_df['id'].astype(str))

# 2. List the actual images downloaded
image_folder = 'test_images_final'
downloaded_images = [f.replace('.jpg', '') for f in os.listdir(image_folder) if f.endswith('.jpg')]
downloaded_ids = set(downloaded_images)

# 3. Compare
missing_images = expected_ids - downloaded_ids
extra_images = downloaded_ids - expected_ids

print(f"--- Data Integrity Report ---")
print(f"Total rows in test2.xlsx: {len(expected_ids)}")
print(f"Total images in folder  : {len(downloaded_ids)}")

if len(missing_images) == 0:
    print("✅ Success: Every row in the Excel file has a matching image.")
else:
    print(f"❌ Warning: {len(missing_images)} images are MISSING.")
    print(f"Sample missing IDs: {list(missing_images)[:5]}")

if len(extra_images) > 0:
    print(f"💡 Note: There are {len(extra_images)} extra images in the folder not in test2.xlsx.")

--- Data Integrity Report ---
Total rows in test2.xlsx: 5396
Total images in folder  : 5396
✅ Success: Every row in the Excel file has a matching image.


In [3]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

def build_vision_model():
    # Use MobileNetV2 with ImageNet weights as you did before
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False 
    
    # Replicate your exact Sequential structure
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1) 
    ])
    
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# Initialize the model
vision_model = build_vision_model()
print("✅ Vision Model is restored and ready.")

✅ Vision Model is restored and ready.


In [4]:
import os
import numpy as np
import pandas as pd

# Load your test data
test_df = pd.read_excel('test2.xlsx')
ids = test_df['id'].values
all_test_features = []

print(f"Extracting features for {len(ids)} images...")

# Loop through in batches of 32 just like your training full_gen
for i in range(0, len(ids), 32):
    batch_ids = ids[i : i + 32]
    batch_imgs = []
    
    for img_id in batch_ids:
        path = os.path.join('test_house_images', f"{img_id}.jpg")
        try:
            # Replicate your exact loading and /255.0 normalization
            img = tf.keras.preprocessing.image.load_img(path, target_size=(224, 224))
            img = tf.keras.preprocessing.image.img_to_array(img) / 255.0
            batch_imgs.append(img)
        except Exception:
            # Replicate your blank image fallback for missing/corrupted files
            batch_imgs.append(np.zeros((224, 224, 3)))
    
    # Process batch through Layer 0 (MobileNet) and Layer 1 (Pooling)
    imgs_array = np.array(batch_imgs)
    x = vision_model.layers[0](imgs_array)
    x = vision_model.layers[1](x)
    
    all_test_features.append(x.numpy())

# Stack into the final 1280-column matrix
visual_features_test = np.vstack(all_test_features)
print(f"✅ Extraction Complete! Shape: {visual_features_test.shape}")

Extracting features for 5404 images...
✅ Extraction Complete! Shape: (5404, 1280)


In [7]:
import pandas as pd

# Convert your 1280-column matrix into a DataFrame
# This matches the 'vis_i' naming convention used in training
test_vis_df = pd.DataFrame(visual_features_test, columns=[f'vis_{i}' for i in range(1280)])

print(f"Success: 'test_vis_df' created with shape {test_vis_df.shape}")

Success: 'test_vis_df' created with shape (5404, 1280)


In [8]:
import joblib

# 1. Load the master list of features the model expects
final_cols = joblib.load('final_feature_list.pkl')

# 2. Identify the expected tabular and visual features
expected_tabular = [f for f in final_cols if not f.startswith('vis_')]
expected_visual = [f for f in final_cols if f.startswith('vis_')]

# 3. Identify what is currently in your DataFrames
current_tabular = set(test_df.columns)
current_visual = set(test_vis_df.columns)

# 4. Find the missing pieces
missing_tabular = [f for f in expected_tabular if f not in current_tabular]
missing_visual = [f for f in expected_visual if f not in current_visual]

print("--- Feature Verification Report ---")
print(f"Total features model expects: {len(final_cols)}")
print(f"Tabular features expected: {len(expected_tabular)}")
print(f"Visual features (Top 20) expected: {len(expected_visual)}")

if not missing_tabular and not missing_visual:
    print("\n✅ SUCCESS: All features are present. You can proceed to prediction.")
else:
    if missing_tabular:
        print(f"\n❌ MISSING TABULAR FEATURES ({len(missing_tabular)}):")
        print(missing_tabular)
        print("Tip: Check your apply_tabular_engineering function for typos.")
    
    if missing_visual:
        print(f"\n❌ MISSING VISUAL FEATURES ({len(missing_visual)}):")
        print(missing_visual)
        print("Tip: Ensure you converted your extracted matrix into a DataFrame with 'vis_i' column names.")

--- Feature Verification Report ---
Total features model expects: 52
Tabular features expected: 32
Visual features (Top 20) expected: 20

❌ MISSING TABULAR FEATURES (14):
['year_sold', 'month_sold', 'living_to_lot_ratio', 'relative_living_size', 'relative_lot_size', 'luxury_index', 'total_rooms', 'avg_room_size', 'house_age', 'years_since_update', 'is_classic', 'bath_per_bed', 'sqft_per_bedroom', 'sqft_grade']
Tip: Check your apply_tabular_engineering function for typos.


In [9]:
# 1. Date-based features (Assuming 'date' column exists in test2.xlsx)
test_df['date'] = pd.to_datetime(test_df['date'])
test_df['year_sold'] = test_df['date'].dt.year
test_df['month_sold'] = test_df['date'].dt.month

# 2. Space and Ratio features
test_df['living_to_lot_ratio'] = test_df['sqft_living'] / test_df['sqft_lot'].replace(0, 1)
test_df['relative_living_size'] = test_df['sqft_living'] / test_df['sqft_living15'].replace(0, 1)
test_df['relative_lot_size'] = test_df['sqft_lot'] / test_df['sqft_lot15'].replace(0, 1)
test_df['sqft_grade'] = test_df['sqft_living'] * test_df['grade']

# 3. Room-based features
test_df['total_rooms'] = test_df['bedrooms'] + test_df['bathrooms']
test_df['avg_room_size'] = test_df['sqft_living'] / test_df['total_rooms'].replace(0, 1)
test_df['bath_per_bed'] = test_df['bathrooms'] / test_df['bedrooms'].replace(0, 1)
test_df['sqft_per_bedroom'] = test_df['sqft_living'] / test_df['bedrooms'].replace(0, 1)

# 4. Quality and Age features (USING YOUR 2015 LOGIC)
test_df['luxury_index'] = test_df['condition'] * test_df['grade']
test_df['house_age'] = 2015 - test_df['yr_built']
test_df['years_since_update'] = 2015 - test_df[['yr_built', 'yr_renovated']].max(axis=1)
test_df['is_classic'] = test_df['yr_built'].apply(lambda x: 1 if x < 1940 else 0)

print("✅ 14 Features engineered successfully with 2015 baseline.")

✅ 14 Features engineered successfully with 2015 baseline.


In [10]:
# 1. Identify which 'vis_' columns the model expects
expected_vis_cols = [f for f in final_cols if f.startswith('vis_')]

# 2. Select ONLY those 20 columns from your 1280-column dataframe
test_vis_selected = test_vis_df[expected_vis_cols]

print(f"Visual Feature Preparation Complete.")
print(f"Original features: {test_vis_df.shape[1]}")
print(f"Selected 'Golden' features: {test_vis_selected.shape[1]}")
print(f"Sample of selected columns: {expected_vis_cols[:5]}")

Visual Feature Preparation Complete.
Original features: 1280
Selected 'Golden' features: 20
Sample of selected columns: ['vis_84', 'vis_111', 'vis_193', 'vis_211', 'vis_276']


In [13]:
import joblib

# Load the trained XGBoost model
final_hybrid_model = joblib.load('honest_hybrid_k20_model.pkl')

In [14]:
# 1. Combine the 32 Tabular + 20 Visual features
# We reset index to ensure rows line up perfectly
X_test_final_combined = pd.concat([test_df.reset_index(drop=True), 
                                  test_vis_selected.reset_index(drop=True)], axis=1)

# 2. Force the columns into the EXACT order the model expects
X_test_final = X_test_final_combined[final_cols]

# 3. Generate Predictions
print("Predicting prices...")
log_preds = final_hybrid_model.predict(X_test_final)

# 4. Reverse the log transformation (np.expm1) to get real prices
test_df['predicted_price'] = np.expm1(log_preds)


Predicting prices...


In [15]:
print(test_df['predicted_price'].head())

0    3.726483e+05
1    8.818920e+05
2    1.107493e+06
3    1.813370e+06
4    7.400080e+05
Name: predicted_price, dtype: float32


In [16]:
# 5. Save the final deliverable
test_df[['id', 'predicted_price']].to_csv('23117040_final.csv', index=False)

print("✅ DONE! Check your folder for 23117040_final.csv")

✅ DONE! Check your folder for 23117040_final.csv
